In [1]:
import numpy as np
import scipy.signal
from matplotlib.pyplot import *
import librosa
import os
import matplotlib.pyplot as plt
import librosa.display
from filter_function import *
from spectrogram import *
from wav_preprocessing import *
import matplotlib.image as mpimg
import glob
from keras.models import Sequential,Model
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D, UpSampling2D, Input,Dropout,Conv2DTranspose,Concatenate
from keras.layers import  Conv1D, MaxPooling1D, UpSampling1D,Concatenate, PReLU
from keras import optimizers

Using TensorFlow backend.


In [2]:
original_dataset_dir = "C:\\Users\\joaro\\OneDrive\\Documents\\deep_learning_jupiter\\free_spoken_digit_dataset\\recordings\\"

base_dir = "C:\\Users\\joaro\\OneDrive\\Documents\\deep_learning_jupiter\\free_spoken_digit_dataset\\super_resolution_re_im"
train_dir = os.path.join(base_dir, 'train\\')

test_dir = os.path.join(base_dir, 'test\\')
train_dir_filtré=os.path.join(base_dir, 'train_dir_filtré\\')
test_dir_filtré=os.path.join(base_dir, 'test_dir_filtré\\')

In [3]:
Xtest,max_test= dir_to_wav_array(test_dir)
Xtrain,max_train= dir_to_wav_array(train_dir)

processing C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\test\
processing C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\train\
C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\train\5_theo_36.wav
C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\train\7_theo_36.wav
C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\train\9_theo_16.wav
C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\train\9_theo_28.wav


In [ ]:
fce=0.04535#sr=22050
ftype="butter"
ordre=4
b,a = creer_filter(fce,"lowpass",ftype,ordre)
filter_path(train_dir,train_dir_filtré,b,a)
filter_path(test_dir,test_dir_filtré,b,a)

In [4]:
print(Xtest.shape)

(200, 8000, 1, 1)


In [5]:
Xtrain_fil,max_train_fil=dir_to_wav_array(train_dir_filtré)
Xtest_fil,max_test_fil=dir_to_wav_array(test_dir_filtré)

processing C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\train_dir_filtré\
C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\train_dir_filtré\5_theo_36.wav
C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\train_dir_filtré\7_theo_36.wav
C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\train_dir_filtré\9_theo_16.wav
C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\train_dir_filtré\9_theo_28.wav
processing C:\Users\joaro\OneDrive\Documents\deep_learning_jupiter\free_spoken_digit_dataset\super_resolution_re_im\test_dir_filtré\


In [79]:
#réseau audio net
input_img = Input(shape=(8000,1,1))

conv1= Conv2D(32, (3, 1), padding='same',strides=1)(input_img)
conv1 = PReLU(shared_axes=[1, 2], name='prelu1')(conv1)
max1= MaxPooling2D((2, 1), padding='same')(conv1)

conv3 = Conv2D(32, (3, 1), padding='same',strides=1)(max1)
conv3 = PReLU(shared_axes=[1, 2], name='prelu3')(conv3)              
encoded = MaxPooling2D((2, 1), padding='same')(conv3)
x = Conv2D(32, (3, 1), padding='same',strides=1)(encoded)
x = PReLU(shared_axes=[1, 2], name='prelu4')(x)


x = UpSampling2D((2, 1))(x)
merge2=Concatenate(axis=-1)([x,conv3])
x = Conv2D(32, (3, 1), padding='same',strides=1)(merge2)
x = PReLU(shared_axes=[1, 2], name='prelu8')(x)
# x=Dropout(0.5)(x)
x = UpSampling2D((2, 1))(x)
# merge1=Concatenate(axis=-1)([x,conv1])
decoded = Conv2D(1, (3, 1), padding='same',strides=1)(x)
decoded = PReLU(shared_axes=[1, 2], name='prelu9')(decoded)
# decoded=Dropout(0.5)(decoded)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=optimizers.Adam(lr=0.001), loss='logcosh')
#optimizer=optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.0, nesterov=False)
autoencoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 8000, 1, 1)   0                                            
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, 8000, 1, 32)  128         input_11[0][0]                   
__________________________________________________________________________________________________
prelu1 (PReLU)                  (None, 8000, 1, 32)  32          conv2d_49[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_21 (MaxPooling2D) (None, 4000, 1, 32)  0           prelu1[0][0]                     
__________________________________________________________________________________________________
conv2d_50 

In [104]:
G=100
autoencoder.fit(Xtrain_fil*G,Xtrain*G,epochs=20,
                batch_size=20,
                shuffle=True,
                validation_data=(Xtest_fil*G, Xtest*G))

Train on 1800 samples, validate on 200 samples
Epoch 1/20
1800/1800 [==============================] - 8s 4ms/step - loss: 0.0482 - val_loss: 0.0313
Epoch 2/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.0445 - val_loss: 0.0307
Epoch 3/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.0436 - val_loss: 0.0300
Epoch 4/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.0443 - val_loss: 0.0294
Epoch 5/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.0430 - val_loss: 0.0293
Epoch 6/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.0413 - val_loss: 0.0283
Epoch 7/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.0405 - val_loss: 0.0275
Epoch 8/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.0410 - val_loss: 0.0287
Epoch 9/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.0389 - val_loss: 0.0265
Epoch 10/20
1800/1800 [============

In [105]:
decoded = autoencoder.predict(Xtest_fil)

In [112]:
p=180
sr=8000
#indice to display
xtest=np.reshape(Xtest[p],(8000))
audio_path_record="C:\\Users\\joaro\\OneDrive\\Documents\\deep_learning_jupiter\\free_spoken_digit_dataset\\"
audio_path_record_in=os.path.join(audio_path_record,"0_jackson_0_wav_test.wav")
soundfile.write(audio_path_record_in,xtest,sr)
ipd.Audio(audio_path_record_in)

In [117]:
print(Xtest[p])

[[[-0.0067097 ]]

 [[-0.01107203]]

 [[-0.00886711]]

 ...

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]]


In [114]:
print(decoded[p])

[[[ 0.03864403]]

 [[ 0.01705245]]

 [[-0.01888631]]

 ...

 [[-0.07647765]]

 [[-0.04984839]]

 [[-0.03538423]]]


In [115]:
y=np.reshape(decoded[p],(8000))
audio_path_record_out=os.path.join(audio_path_record,"0_jackson_0_wav_reconstructed.wav")
soundfile.write(audio_path_record_out,y,sr)
ipd.Audio(audio_path_record_out)

In [116]:
xtest_fil=np.reshape(Xtest_fil[p],(8000))
audio_path_record_in_fil=os.path.join(audio_path_record,"0_jackson_0_filtered.wav")
soundfile.write(audio_path_record_in_fil,xtest_fil,sr)
ipd.Audio(audio_path_record_in_fil)

In [111]:
import sklearn
error=sklearn.metrics.mean_squared_error(xtest,y)
print(error)

2.3181293763254806e-06
